__Kaggle competition - Titanic survivorship__

1. [Import](#Import)
    1. [Tools](#Tools)
    1. [Data](#Data)    
1. [EDA](#EDA)
    1. [Categorical feature EDA](#Categorical-feature-EDA)
    1. [numeric feature EDA](#numeric-feature-EDA)
    1. [Faceting](#Faceting)
    1. [Target variable evaluation](#Target-variable-evaluation)    
1. [Data preparation](#Data-preparation)
    1. [Missing data](#Missing-data)
    1. [Engineering](#Engineering)
    1. [Encoding](#Encoding)
    1. [Transformation](#Transformation)
        1. [Polynomial features](#Polynomial-features)
        1. [Skew](#Skew)
        1. [Scale](#Scale)
    1. [Outliers](#Outliers)
1. [Feature importance](#Feature-importance)    
1. [Modeling](#Modeling)
    1. [Data preparation](#Data-preparation-1)
    1. [Bayesian hyper-parameter optimization](#Bayesian-hyper-parameter-optimization)
    1. [Model performance evaluation - standard models](#Model-performance-evaluation-standard-models)
    1. [Model explanability](#Model-explanability)
    1. [Submission - standard models](#Submission-standard-models)
1. [Stacking](#Stacking)
    1. [Primary models](#Primary-models)
    1. [Meta model](#Meta-model)                
    1. [Model performance evaluation - stacked models](#Model-performance-evaluation-stacked-models)
    1. [Submission - stacked models](#Submission-stacked-models)    

# Import

<a id = 'Import'></a>

## Tools

<a id = 'Tools'></a>

In [ ]:
# standard libary and settings
import os
import sys
import importlib
import itertools
from functools import reduce
import time; rundate = time.strftime("%Y%m%d")

import warnings
warnings.simplefilter("ignore")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# data extensions and settings
import numpy as np
np.set_printoptions(threshold=np.inf, suppress=True)

import pandas as pd
pd.set_option("display.max_rows", 500); pd.set_option("display.max_columns", 500)
pd.options.display.float_format = "{:,.6f}".format

# modeling extensions
import sklearn.base as base
import sklearn.datasets as datasets
import sklearn.ensemble as ensemble
import sklearn.impute as impute
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing

import eif
import shap
shap.initjs()
from eli5.sklearn import PermutationImportance
from pdpbox import pdp, get_dataset, info_plots

# visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

%matplotlib inline

try:
    #     import mlmachine as mlm
    #     from prettierplot.plotter import PrettierPlot
    #     import prettierplot.style as style
    import asdfasd
except ModuleNotFoundError:
    sys.path.append(
        "../../../mlmachine"
    ) if "../../../../mlmachine" not in sys.path else None
    sys.path.append(
        "../../../prettierplot"
    ) if "../../../../prettierplot" not in sys.path else None

    import mlmachine as mlm
    from mlmachine.features.preprocessing import (
        DataFrameSelector,
        PlayWithPandas,
        UnprocessedColumnAdder,
        ContextImputer,
        PandasFeatureUnion,
        DualTransformer,
    )
    from prettierplot.plotter import PrettierPlot
    import prettierplot.style as style
else:
    print(
        "This notebook relies on the libraries mlmachine and prettierplot. Please run:"
    )
    print("\tpip install mlmachine")
    print("\tpip install prettierplot")

## Data

<a id = 'Data'></a>

In [ ]:
# load data and print dimensions
dfTrain = pd.read_csv("s3://tdp-ml-datasets/kaggle-titanic/train.csv")
dfValid = pd.read_csv("s3://tdp-ml-datasets/kaggle-titanic/test.csv")

print("Training data dimensions: {}".format(dfTrain.shape))
print("Validation data dimensions: {}".format(dfValid.shape))

In [ ]:
# display info and first 5 rows
dfTrain.info()
display(dfTrain[:5])

In [ ]:
# review counts of different column types
dfTrain.dtypes.value_counts()

In [ ]:
# Load training data into mlmachine
train = mlm.Machine(
    data=dfTrain,
    target="Survived",
    removeFeatures=["PassengerId", "Ticket"],
    overrideCat=["Pclass"],
    targetType="categorical",
)
print(train.data.shape)

In [ ]:
# load training data into mlmachine
valid = mlm.Machine(
    data=dfValid,
    removeFeatures=["PassengerId", "Ticket"],
    overrideCat=["Pclass"],
)
print(valid.data.shape)

# EDA

<a id = 'EDA'></a>

## Categorical feature EDA

<a id = 'Categorical-feature-EDA'></a>

##### Univariate & feature vs. target

In [ ]:
# categorical features
for feature in train.featureType["categorical"]:
    train.edaCatTargetCatFeat(feature=feature, levelCountCap=50)

## numeric feature EDA

<a id = 'numeric-feature-EDA'></a>

##### Univariate & feature vs. target

In [ ]:
# numeric features
for feature in train.featureType["numeric"]:
    train.edaCatTargetNumFeat(feature=feature)

##### Correlation

###### Correlation (all samples)

In [ ]:
# correlation heat map
p = PrettierPlot()
ax = p.makeCanvas()
p.prettyCorrHeatmap(df=train.data, annot=True, ax=ax)

###### Correlation (top vs. target)

In [ ]:
# correlation heat map with most highly correlated features relative to the target
p = PrettierPlot(plotOrientation='tall',chartProp=10)
ax = p.makeCanvas()
p.prettyCorrHeatmapTarget(
    df=train.data, target=train.target, thresh=0.01, annot=True, ax=ax
)

##### Pair plot

<a id = 'Pair-plot'></a>

In [ ]:
# # pair plot
# p = PrettierPlot(chartProp=12)
# p.prettyPairPlot(df=train.data[["Age","Fare","Sex","Pclass","Embarked"]].dropna(), diag_kind="auto")

In [ ]:
# # pair plot
# p = PrettierPlot(chartProp=12)
# p.prettyPairPlot(
#     df=train.data.dropna(),
#     diag_kind="kde",
#     target=train.target,
#     cols=["Age", "Fare", "Pclass", "Parch", "SibSp"],
#     legendLabels=["Died", "Survived"],
#     bbox=(2.0, 0.0),
# )

## Faceting

<a id = 'Faceting'></a>

##### Categorical by categorical

In [ ]:
# facet Pclass vs Embarked
p = PrettierPlot(chartProp=12)
ax = p.makeCanvas(title="Survivorship, embark location by passenger class", yShift=0.7)
p.prettyFacetTwoCatBar(
    df=train.recombineData(train.data, train.target),
    x="Embarked",
    y=train.target.name,
    split="Pclass",
    yUnits="ff",
    ax=ax,
)

In [ ]:
# facet Pclass vs Embarked
p = PrettierPlot(chartProp=12)
ax = p.makeCanvas(title="Survivorship, passenger class by gender", yShift=0.7)
p.prettyFacetTwoCatBar(
    df=train.recombineData(train.data, train.target),
    x="Pclass",
    y=train.target.name,
    split="Sex",
    yUnits="ff",
    ax=ax,
)

In [ ]:
# facet Pclass vs Embarked
p = PrettierPlot(chartProp=12)
ax = p.makeCanvas(title="Survivorship,embark location by gender", yShift=0.7)
p.prettyFacetTwoCatBar(
    df=train.recombineData(train.data, train.target),
    x="Embarked",
    y=train.target.name,
    split="Sex",
    yUnits="ff",
    ax=ax,
)

In [ ]:
#
p = PrettierPlot()
p.prettyFacetTwoCatPoint(
    df=train.recombineData(train.data, train.target),
    x="Sex",
    y=train.target.name,
    split="Pclass",
    catCol="Embarked",
    aspect=1.0,
    height=5,
    bbox=(1.3, 1.2),
    legendLabels=["1st class", "2nd class", "3rd class"],
)

In [ ]:
#
p = PrettierPlot()
p.prettyFacetTwoCatPoint(
    df=train.recombineData(train.data, train.target).dropna(subset=["Embarked"]),
    x="Embarked",
    y=train.target.name,
    split="Pclass",
    catCol="Sex",
    aspect=1.0,
    height=5,
    bbox=(1.5, 0.8),
    legendLabels=["1st class", "2nd class", "3rd class"],
)

##### Categorical by numeric

In [ ]:
#
p = PrettierPlot()
p.prettyFacetCatNumHist(
    df=train.recombineData(train.data, train.target),
    split=train.target.name,
    legendLabels=["Died", "Lived"],
    catRow="Sex",
    catCol="Embarked",
    numCol="Age",
    bbox=(1.9, 1.0),
    height=4,
    aspect=1,
)

In [ ]:
#
p = PrettierPlot(chartProp=15)
p.prettyFacetCatNumScatter(
    df=train.recombineData(train.data, train.target),
    split=train.target.name,
    legendLabels=["Died", "Lived"],
    catRow="Sex",
    catCol="Embarked",
    x="Fare",
    y="Age",
    bbox=(1.9, 1.0),
    height=4,
    aspect=1,
)

## Target variable evaluation

<a id = 'Target-variable-evaluation'></a>

In [ ]:
# null score
pd.Series(train.target).value_counts(normalize=True)

# Data preparation

<a id = 'Data-preparation'></a>

## Missing data


<a id = 'Missing-data'></a>

##### Training

In [ ]:
# evaluate missing data
train.edaMissingSummary()

In [ ]:
# missingno matrix
msno.matrix(train.data)

In [ ]:
# missingno bar
msno.bar(train.data)

In [ ]:
# missingno heatmap
msno.heatmap(train.data)

In [ ]:
# missingno dendrogram
msno.dendrogram(train.data)

##### Validation

In [ ]:
# evaluate missing data
valid.edaMissingSummary()

In [ ]:
# missingno matrix
msno.matrix(valid.data)

In [ ]:
# missingno bar
msno.bar(valid.data)

In [ ]:
# missingno heatmap
msno.heatmap(valid.data)

In [ ]:
# missingno dendrogram
msno.dendrogram(valid.data)

##### Training vs. validation


In [ ]:
# compare feature with missing data
train.missingColCompare(train=train.data, validation=valid.data)

##### Impute

In [ ]:
# impute pipeline
imputePipe = PandasFeatureUnion([
    ("age", pipeline.make_pipeline(
        DataFrameSelector(["Age","Pclass"]),
        ContextImputer(nullCol="Age", contextCol="Pclass", strategy="mean")
    )),
    ("fare", pipeline.make_pipeline(
        DataFrameSelector(["Fare","Pclass"]),
        ContextImputer(nullCol="Fare", contextCol="Pclass")
    )),
    ("embarked", pipeline.make_pipeline(
        DataFrameSelector(["Embarked"]),
        PlayWithPandas(impute.SimpleImputer(strategy="most_frequent"))
    )),
    ("cabin", pipeline.make_pipeline(
        DataFrameSelector(["Cabin"]),
        PlayWithPandas(impute.SimpleImputer(strategy="constant", fill_value="X"))
    )),
    ("diff", pipeline.make_pipeline(
        DataFrameSelector(list(set(train.data.columns).difference(["Age","Fare","Embarked","Cabin"]))),
    )),
])

train.data = imputePipe.fit_transform(train.data)
valid.data = imputePipe.transform(valid.data)

In [ ]:
train.edaMissingSummary()

In [ ]:
valid.edaMissingSummary()

## Engineering

<a id = 'Engineering'></a>

##### Training

In [ ]:
# parse titles to learn gender, and identify rare titles which may convey status
title = [i.split(",")[1].split(".")[0].strip() for i in train.data["Name"]]
train.data["Title"] = pd.Series(
    title,
    index=train.data.index,
    dtype="category",
)
train.data["Title"] = train.data["Title"].replace(
    [
        "Lady",
        "the Countess",
        "Countess",
        "Capt",
        "Col",
        "Don",
        "Dr",
        "Major",
        "Rev",
        "Sir",
        "Jonkheer",
        "Dona",
    ],
    "Rare",
)
train.data["Title"] = train.data["Title"].map(
    {"Master": 0, "Miss": 1, "Ms": 1, "Mme": 1, "Mlle": 1, "Mrs": 1, "Mr": 2, "Rare": 3}
)
train.data["Title"] = train.data["Title"].astype("category")

# distill cabin feature
train.data["CabinQuarter"] = pd.Series(
    [i[0] if not pd.isnull(i) else "X" for i in train.data["Cabin"]],
    index=train.data.index,
    dtype="category",
)

# family size features and binning
train.data["FamilySize"] = train.data["SibSp"] + train.data["Parch"] + 1
train.data["FamilySize"] = train.data["FamilySize"].astype("int64")

# update featureType and drop columns
train.featureTypeUpdate(columnsToDrop=["Name","Cabin"])

##### Validation

In [ ]:
# parse titles to learn gender, and identify rare titles which may convey status
title = [i.split(",")[1].split(".")[0].strip() for i in valid.data["Name"]]
valid.data["Title"] = pd.Series(
    title,
    index=valid.data.index,
    dtype="category",
)
valid.data["Title"] = valid.data["Title"].replace(
    [
        "Lady",
        "the Countess",
        "Countess",
        "Capt",
        "Col",
        "Don",
        "Dr",
        "Major",
        "Rev",
        "Sir",
        "Jonkheer",
        "Dona",
    ],
    "Rare",
)
valid.data["Title"] = valid.data["Title"].map(
    {"Master": 0, "Miss": 1, "Ms": 1, "Mme": 1, "Mlle": 1, "Mrs": 1, "Mr": 2, "Rare": 3}
)
valid.data["Title"] = valid.data["Title"].astype("category")

# distill cabin feature
valid.data["CabinQuarter"] = pd.Series(
    [i[0] if not pd.isnull(i) else "X" for i in valid.data["Cabin"]],
    index=valid.data.index,
    dtype="category",
)

# additional features
valid.data["FamilySize"] = valid.data["SibSp"] + valid.data["Parch"] + 1
valid.data["FamilySize"] = valid.data["FamilySize"].astype("int64")

# update featureType and drop columns
valid.featureTypeUpdate(columnsToDrop=["Name","Cabin"])

## Encoding

<a id = 'Encoding'></a>

##### Training

In [ ]:
# counts of unique values in training data string columns
train.data[train.featureType["categorical"]].apply(pd.Series.nunique, axis=0)

In [ ]:
# print unique values in each categorical columns
for col in train.data[train.featureType["categorical"]]:
    print(col, np.unique(train.data[col]))

##### Validation

In [ ]:
# counts of unique values in validation data string columns
valid.data[valid.featureType["categorical"]].apply(pd.Series.nunique, axis=0)

In [ ]:
# print unique values in each categorical columns
for col in valid.data[valid.featureType["categorical"]]:
    print(col, np.unique(valid.data[col]))

##### Training vs. validation

In [ ]:
# identify values that are present in the training data but not the validation data, and vice versa
for col in train.featureType["categorical"]:
    if col not in ["Name", "Cabin"]:
        trainValues = train.data[col].unique()
        validValues = valid.data[col].unique()

        trainDiff = set(trainValues) - set(validValues)
        validDiff = set(validValues) - set(trainValues)

        if len(trainDiff) > 0 or len(validDiff) > 0:
            print("\n\n*** " + col)
            print("Value present in training data, not in validation data")
            print(trainDiff)
            print("Value present in validation data, not in training data")
            print(validDiff)

##### Encode

In [ ]:
#
nominalColumns = ["Embarked","Sex","CabinQuarter","Title"]

ordinalColumns = ["Pclass"]
ordinalEncodings = [
        [0, 1, 2, 3], # Pclass 
    ]

# encode pipeline
encodePipe = PandasFeatureUnion([
    ("ordinal", pipeline.make_pipeline(
        DataFrameSelector(ordinalColumns),
        PlayWithPandas(preprocessing.OrdinalEncoder(categories=ordinalEncodings)),
    )),
    ("nominal", pipeline.make_pipeline(
        DataFrameSelector(nominalColumns),
        PlayWithPandas(preprocessing.OneHotEncoder(drop="first")),
    )),
    ("diff", pipeline.make_pipeline(
        DataFrameSelector(list(set(train.data.columns).difference(nominalColumns + ordinalColumns))),
    )),
])

train.data = encodePipe.fit_transform(train.data)
valid.data = encodePipe.transform(valid.data)

train.featureTypeUpdate()
valid.featureTypeUpdate()

## Transformation

<a id = 'Transformation'></a>

### Polynomial features

<a id = 'Polynomial-features'></a>

##### Transform

In [ ]:
# transform pipe
polynomialPipe = PandasFeatureUnion([
    ("polynomial", pipeline.make_pipeline(
        DataFrameSelector(train.featureType["numeric"]),
        PlayWithPandas(preprocessing.PolynomialFeatures(degree=2, interaction_only=False, include_bias=False))
    )),
    ("diff", pipeline.make_pipeline(
        DataFrameSelector(list(set(train.data.columns).difference(train.featureType["numeric"]))),
    )),
])

train.data = polynomialPipe.fit_transform(train.data)
valid.data = polynomialPipe.transform(valid.data)

train.featureTypeUpdate()
valid.featureTypeUpdate()

### Skew

<a id = 'Skew'></a>

##### Training

In [ ]:
# evaluate skew of numeric features - training data
train.skewSummary()

##### Validation

In [ ]:
# evaluate skew of numeric features - validation data
valid.skewSummary()

##### Transform

In [ ]:
# skew correction pipeline
skewPipe = PandasFeatureUnion([
    ("skew", pipeline.make_pipeline(
        DataFrameSelector(train.featureType["numeric"]),
        DualTransformer(),
    )),    
    ("diff", pipeline.make_pipeline(
        DataFrameSelector(list(set(train.data.columns).difference(train.featureType["numeric"]))),
    )),
])

train.data = skewPipe.fit_transform(train.data)
valid.data = skewPipe.transform(valid.data)

train.featureTypeUpdate()
valid.featureTypeUpdate()

### Scale

<a id = 'Scale'></a>

##### Transform

In [ ]:
#
scalePipe = PandasFeatureUnion([
    ("scale", pipeline.make_pipeline(
        DataFrameSelector(train.featureType["numeric"]),
        PlayWithPandas(preprocessing.StandardScaler())
    )),
    ("diff", pipeline.make_pipeline(
        DataFrameSelector(list(set(train.data.columns).difference(train.featureType["numeric"]))),
    )),
])

train.data = scalePipe.fit_transform(train.data)
valid.data = scalePipe.transform(valid.data)

train.featureTypeUpdate()
valid.featureTypeUpdate()

## Outliers


<a id = 'Outliers'></a>

In [ ]:
# identify outliers using IQR
trainPipe = pipeline.Pipeline([
    ("outlier",train.OutlierIQR(
                outlierCount=25,
                iqrStep=1.5,
                features=train.featureType["numeric"],
                dropOutliers=False,))
    ])
train.data = trainPipe.transform(train.data)

# capture outliers
iqrOutliers = np.array(sorted(trainPipe.named_steps["outlier"].outliers_))
print(iqrOutliers)

In [ ]:
# identify outliers using Isolation Forest
clf = ensemble.IsolationForest(
    behaviour="new", max_samples=train.data.shape[0], random_state=0, contamination=0.01
)
clf.fit(train.data[train.data.columns])
preds = clf.predict(train.data[train.data.columns])

# evaluate index values
mask = np.isin(preds, -1)
ifOutliers = np.array(train.data[mask].index)
print(ifOutliers)

In [ ]:
# identify outliers using extended isolation forest
trainPipe = pipeline.Pipeline([
    ("outlier",train.ExtendedIsoForest(
                cols=train.featureType["numeric"],
                nTrees=100,
                sampleSize=256,
                ExtensionLevel=1,
                anomaliesRatio=0.03,
                dropOutliers=False,))
    ])
train.data = trainPipe.transform(train.data)

# capture outliers
eifOutliers = np.array(sorted(trainPipe.named_steps["outlier"].outliers_))
print(eifOutliers)

In [ ]:
# identify outliers that are identified in multiple algorithms
outliers = reduce(np.intersect1d, (iqrOutliers, ifOutliers, eifOutliers))
# outliers = reduce(np.intersect1d, (ifOutliers, eifOutliers))
print(outliers)

In [ ]:
# review outlier identification summary
outlierSummary = train.outlierSummary(iqrOutliers=iqrOutliers,
                             ifOutliers=ifOutliers,
                             eifOutliers=eifOutliers
                            )
outlierSummary[outlierSummary["Count"] >= 3]

In [ ]:
# remove outlers from predictors and response
outliers = np.array([27, 88, 258, 311, 341, 438, 679, 737, 742])
train.data = train.data.drop(outliers)
train.target = train.target.drop(index=outliers)

# Feature importance

<a id = 'Feature-importance'></a>

In [ ]:
# generate feature importance summary
estimators = [
    "lightgbm.LGBMClassifier",
    "ensemble.RandomForestClassifier",
    "ensemble.GradientBoostingClassifier",
    "ensemble.ExtraTreesClassifier",
    "ensemble.AdaBoostClassifier",
    "xgboost.XGBClassifier",
]

fs = train.FeatureSelector(
    data=train.data, target=train.target, estimators=estimators, rank=True
)
fs.featureSelectorSummary()

In [ ]:
# calculate cross-validation performance
estimators = [
    "svm.SVC",
    "lightgbm.LGBMClassifier",
    "linear_model.LogisticRegression",
    "xgboost.XGBClassifier",
    "ensemble.RandomForestClassifier",
    "ensemble.GradientBoostingClassifier",
    #"ensemble.AdaBoostClassifier",
    "ensemble.ExtraTreesClassifier",
    "neighbors.KNeighborsClassifier",
]

cvSummary = fs.featureSelectorCrossVal(
    metrics=["accuracy"],
    nFolds=8,
    step=1
)

###### Accuracy

In [ ]:
# visualize CV performance for diminishing feature set
fs.featureSelectorResultsPlot(
    metric="accuracy",
    titleScale=0.8,
)

In [ ]:
df = fs.featuresUsedSummary(metric="accuracy")
df

In [ ]:
# list feature that showed up in at least X models
df[df["count"] >= 7].index

# Modeling

<a id = 'Modeling'></a>

## Data preparation

<a id = 'Data-preparation-1'></a>

##### Prepare data

In [ ]:
#################################################################################
# import training data
dfTrain = pd.read_csv("s3://tdp-ml-datasets/kaggle-titanic/train.csv")
train = mlm.Machine(
    data=dfTrain,
    target="Survived",
    removeFeatures=["PassengerId", "Ticket"],
    overrideCat=["Pclass"],
    targetType="categorical",
)

# import validation data
dfValid = pd.read_csv("s3://tdp-ml-datasets/kaggle-titanic/test.csv")
valid = mlm.Machine(
    data=dfValid,
    removeFeatures=["PassengerId", "Ticket"],
    overrideCat=["Pclass"],
)

#################################################################################
# impute pipeline
imputePipe = PandasFeatureUnion([
    ("age", pipeline.make_pipeline(
        DataFrameSelector(["Age","Pclass"]),
        ContextImputer(nullCol="Age", contextCol="Pclass", strategy="mean")
    )),
    ("fare", pipeline.make_pipeline(
        DataFrameSelector(["Fare","Pclass"]),
        ContextImputer(nullCol="Fare", contextCol="Pclass")
    )),
    ("embarked", pipeline.make_pipeline(
        DataFrameSelector(["Embarked"]),
        PlayWithPandas(impute.SimpleImputer(strategy="most_frequent"))
    )),
    ("cabin", pipeline.make_pipeline(
        DataFrameSelector(["Cabin"]),
        PlayWithPandas(impute.SimpleImputer(strategy="constant", fill_value="X"))
    )),
    ("diff", pipeline.make_pipeline(
        DataFrameSelector(list(set(train.data.columns).difference(["Age","Fare","Embarked","Cabin"]))),
    )),
])

train.data = imputePipe.fit_transform(train.data)
valid.data = imputePipe.transform(valid.data)

#################################################################################
# feature engineering - training

# parse titles to learn gender, and identify rare titles which may convey status
title = [i.split(",")[1].split(".")[0].strip() for i in train.data["Name"]]
train.data["Title"] = pd.Series(
    title,
    index=train.data.index,
    dtype="category",
)
train.data["Title"] = train.data["Title"].replace(
    [
        "Lady",
        "the Countess",
        "Countess",
        "Capt",
        "Col",
        "Don",
        "Dr",
        "Major",
        "Rev",
        "Sir",
        "Jonkheer",
        "Dona",
    ],
    "Rare",
)
train.data["Title"] = train.data["Title"].map(
    {"Master": 0, "Miss": 1, "Ms": 1, "Mme": 1, "Mlle": 1, "Mrs": 1, "Mr": 2, "Rare": 3}
)
train.data["Title"] = train.data["Title"].astype("category")

# distill cabin feature
train.data["CabinQuarter"] = pd.Series(
    [i[0] if not pd.isnull(i) else "X" for i in train.data["Cabin"]],
    index=train.data.index,
    dtype="category",
)

# family size features
train.data["FamilySize"] = train.data["SibSp"] + train.data["Parch"] + 1
train.data["FamilySize"] = train.data["FamilySize"].astype("int64")

train.featureTypeUpdate(columnsToDrop=["Name","Cabin"])

#################################################################################
# feature engineering - validation

# parse titles to learn gender, and identify rare titles which may convey status
title = [i.split(",")[1].split(".")[0].strip() for i in valid.data["Name"]]
valid.data["Title"] = pd.Series(
    title,
    index=valid.data.index,
    dtype="category"
)
valid.data["Title"] = valid.data["Title"].replace(
    [
        "Lady",
        "the Countess",
        "Countess",
        "Capt",
        "Col",
        "Don",
        "Dr",
        "Major",
        "Rev",
        "Sir",
        "Jonkheer",
        "Dona",
    ],
    "Rare",
)
valid.data["Title"] = valid.data["Title"].map(
    {"Master": 0, "Miss": 1, "Ms": 1, "Mme": 1, "Mlle": 1, "Mrs": 1, "Mr": 2, "Rare": 3}
)
valid.data["Title"] = valid.data["Title"].astype("category")

# distill cabin feature
valid.data["CabinQuarter"] = pd.Series(
    [i[0] if not pd.isnull(i) else "X" for i in valid.data["Cabin"]],
    index=valid.data.index,
    dtype="category",
)

# additional features
valid.data["FamilySize"] = valid.data["SibSp"] + valid.data["Parch"] + 1
valid.data["FamilySize"] = valid.data["FamilySize"].astype("int64")

valid.featureTypeUpdate(columnsToDrop=["Name","Cabin"])

#################################################################################
# feature transformation pipeline
nominalColumns = ["Embarked","Sex","CabinQuarter","Title"]

ordinalColumns = ["Pclass"]
ordinalEncodings = [
        [0, 1, 2, 3], # Pclass 
    ]

transformPipe = PandasFeatureUnion([
    ("ordinal", pipeline.make_pipeline(
        DataFrameSelector(ordinalColumns),
        PlayWithPandas(preprocessing.OrdinalEncoder(categories=ordinalEncodings)),
    )),
    ("nominal", pipeline.make_pipeline(
        DataFrameSelector(nominalColumns),
        PlayWithPandas(preprocessing.OneHotEncoder(drop="first")),
    )),
    ("numeric", pipeline.make_pipeline(
        DataFrameSelector(train.featureType["numeric"]),
        PlayWithPandas(preprocessing.PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)),
        DualTransformer(),
        PlayWithPandas(preprocessing.StandardScaler())
    )),
    ("diff", pipeline.make_pipeline(
        DataFrameSelector(list(set(train.data.columns).difference(nominalColumns + ordinalColumns + train.featureType["numeric"]))),
    )),
])

train.data = transformPipe.fit_transform(train.data)
valid.data = transformPipe.transform(valid.data)

train.featureTypeUpdate()
valid.featureTypeUpdate()

#################################################################################
# remove outliers
outliers = np.array([27, 88, 258, 311, 341, 438, 679, 737, 742])
train.data = train.data.drop(outliers)
train.target = train.target.drop(index=outliers)

# # accuracy >= 8
# bestCols = ['Age*Fare','Title_2','Fare*FamilySize','Pclass','Sex_male','Fare_yj']# accuracy >= 5

# train.data = train.data[bestCols]
# valid.data = valid.data[bestCols]

# # print('completed')

## Bayesian hyper-parameter optimization

<a id = 'Bayesian-hyper-parameter-optimization'></a>

In [ ]:
# parameter space
allSpace = {
    "lightgbm.LGBMClassifier": {
        "class_weight": hp.choice("class_weight", [None]),
        "colsample_bytree": hp.uniform("colsample_bytree", 0.4, 0.7),
        "boosting_type": hp.choice("boosting_type", ["dart"]),
        "subsample": hp.uniform("subsample", 0.5, 1),
        "learning_rate": hp.uniform("learning_rate", 0.15, 0.25),
        "max_depth": hp.choice("max_depth", np.arange(4, 20, dtype=int)),
        "min_child_samples": hp.quniform("min_child_samples", 50, 150, 5),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "num_leaves": hp.quniform("num_leaves", 30, 70, 1),
        "reg_alpha": hp.uniform("reg_alpha", 0.75, 1.25),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "subsample_for_bin": hp.quniform("subsample_for_bin", 100000, 350000, 20000),
    },
    "linear_model.LogisticRegression": {
        "C": hp.uniform("C", 0.04, 0.1),
        "penalty": hp.choice("penalty", ["l1"]),
    },
    "xgboost.XGBClassifier": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.4, 0.7),
        "gamma": hp.quniform("gamma", 0.0, 10, 0.05),
        "learning_rate": hp.quniform("learning_rate", 0.01, 0.2, 0.01),
        "max_depth": hp.choice("max_depth", np.arange(2, 15, dtype=int)),
        "min_child_weight": hp.quniform("min_child_weight", 2.5, 7.5, 1),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "subsample": hp.uniform("subsample", 0.4, 0.7),
    },
    "ensemble.RandomForestClassifier": {
        "bootstrap": hp.choice("bootstrap", [True, False]),
        "max_depth": hp.choice("max_depth", np.arange(2, 10, dtype=int)),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 8000, 10, dtype=int)),
        "max_features": hp.choice("max_features", ["sqrt"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(15, 25, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 20, dtype=int)),
    },
    "ensemble.GradientBoostingClassifier": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 11, dtype=int)),
        "max_features": hp.choice("max_features", ["sqrt"]),
        "learning_rate": hp.quniform("learning_rate", 0.01, 0.09, 0.01),
        "loss": hp.choice("loss", ["deviance", "exponential"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "ensemble.AdaBoostClassifier": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "learning_rate": hp.quniform("learning_rate", 0.1, 0.25, 0.01),
        "algorithm": hp.choice("algorithm", ["SAMME"]),
    },
    "ensemble.ExtraTreesClassifier": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 15, dtype=int)),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(4, 30, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 20, dtype=int)),
        "max_features": hp.choice("max_features", ["auto"]),
        "criterion": hp.choice("criterion", ["entropy"]),
    },
    "svm.SVC": {
        "C": hp.uniform("C", 4, 15),
        "decision_function_shape": hp.choice("decision_function_shape", ["ovr"]),
        "gamma": hp.uniform("gamma", 0.00000001, 1.5),
    },
    "neighbors.KNeighborsClassifier": {
        "algorithm": hp.choice("algorithm", ["ball_tree", "brute"]),
        "n_neighbors": hp.choice("n_neighbors", np.arange(1, 15, dtype=int)),
        "weights": hp.choice("weights", ["uniform"]),
    },
}

In [ ]:
# execute bayesian optimization grid search
analysis = "titanic"
train.execBayesOptimSearch(
    allSpace=allSpace,
    resultsDir="{}_hyperopt_{}.csv".format(rundate, analysis),
    X=train.data,
    y=train.target,
    scoring="accuracy",
    nFolds=2,
    nJobs=4,
    iters=8,
    verbose=0,
)

##### Model loss by iteration

In [ ]:
# read scores summary table
analysis = "titanic"
rundate = "20190807"
bayesOptimSummary = pd.read_csv(
    "{}_hyperopt_{}.csv".format(rundate, analysis), na_values="nan"
)
bayesOptimSummary[:5]

In [ ]:
# model loss plot
for estimator in np.unique(bayesOptimSummary["estimator"]):
    train.modelLossPlot(bayesOptimSummary=bayesOptimSummary, estimator=estimator)

##### Parameter selection by iteration

In [ ]:
# estimator parameter plots
for estimator in np.unique(bayesOptimSummary["estimator"]):
    train.modelParamPlot(
        bayesOptimSummary=bayesOptimSummary,
        estimator=estimator,
        allSpace=allSpace,
        nIter=100,
#         chartProp=15,
        titleScale=0.8
    )

In [ ]:
sampleSpace = {
                'param': hp.uniform('param', np.log(0.4), np.log(0.6))
#     "": 0.000001 + hp.uniform("gamma", 0.000001, 10)
    #             'param2': hp.loguniform('param2', np.log(0.001), np.log(0.01))
}

train.samplePlot(sampleSpace, 1000)

In [ ]:
# pair-wise comparison
p = PrettierPlot(chartProp=12)
p.prettyPairPlotCustom(
    df=train.unpackBayesOptimSummary(bayesOptimSummary, "lightgbm.LGBMClassifier"),
    cols=["colsample_bytree", "learning_rate", "iteration","iterLoss"],
    gradientCol="iteration",
    color=style.styleGrey
)


## Model performance evaluation - standard models

<a id = 'Model-performance-evaluation-standard-models'></a>

In [ ]:
topModels = train.topBayesOptimModels(bayesOptimSummary=bayesOptimSummary, numModels=1)
topModels

In [ ]:
# classification panel, single model
estimator = "svm.SVC"
modelIter = 135
# estimator = 'ensemble.GradientBoostingClassifier'; modelIter = 590
# estimator = 'xgboost.XGBClassifier'; modelIter = 380

model = train.BayesOptimModelBuilder(
    bayesOptimSummary=bayesOptimSummary, estimator=estimator, modelIter=modelIter
)

train.classificationPanel(
    model=model,
    XTrain=train.data,
    yTrain=train.target,
    cmLabels=["Dies", "Survives"],
    nFolds=5,
    titleScale=0.9
)

In [ ]:
# create classification reports for training data
for estimator, modelIters in topModels.items():
    for modelIter in modelIters:
        model = train.BayesOptimModelBuilder(
            bayesOptimSummary=bayesOptimSummary,
            estimator=estimator,
            modelIter=modelIter,
        )
        train.classificationPanel(
            model=model, XTrain=train.data, yTrain=train.target, cmLabels=['Dies', 'Survives'], nFolds=4
        )

## Model explanability

<a id = 'Feature-importance'></a>

In [ ]:
# 
# estimator = "ensemble.ExtraTreesClassifier"; modelIter = 145
# estimator = "svm.SVC"; modelIter = 135
estimator = "ensemble.GradientBoostingClassifier"; modelIter = 490

model = train.BayesOptimModelBuilder(
    bayesOptimSummary=bayesOptimSummary, estimator=estimator, modelIter=modelIter
)
model.fit(train.data.values, train.target.values)

##### Permutation importance

In [ ]:
# permutation importance - how much does performance decrease when shuffling a certain feature?
perm = PermutationImportance(model.model, random_state=1).fit(train.data, train.target)
eli5.show_weights(perm, feature_names=train.data.columns.tolist())

##### SHAP values - training data

###### Force plots - single observations

In [ ]:
# SHAP force plots for individual observations
for i in train.data.index[:2]:
    train.singleShapVizTree(obsIx=i, model=model, data=train.data, target=train.target)

###### Force plots - multiple observations

In [ ]:
# SHAP force plot a set of data
visual = train.multiShapVizTree(obsIxs=train.data.index, model=model, data=train.data)
visual

###### Dependence plots

In [ ]:
# generate SHAP values for set of observations
obsData, _, obsShapValues = train.multiShapValueTree(
    obsIxs=train.data.index, model=model, data=train.data
)

In [ ]:
# SHAP dependence plot grid
gridFeatures = ["Pclass", "Age", "Fare", "SibSp","Parch"]

train.shapDependenceGrid(
    obsData=obsData,
    obsShapValues=obsShapValues,
    gridFeatures=gridFeatures,
    allFeatures=train.data.columns,
    dotSize=35,
    alpha=0.5,
)

In [ ]:
# single SHAP dependence plot
p = PrettierPlot()
ax = p.makeCanvas()

train.shapDependencePlot(
    obsData=obsData,
    obsShapValues=obsShapValues,
    scatterFeature="Age",
    colorFeature="Parch",
    featureNames=train.data.columns,
    dotSize=50,
    alpha=0.5,
    ax=ax
)


In [ ]:
# SHAP dependence plots for all feature relative to an interaction feature
featureNames = train.data.columns.tolist()
topShap = np.argsort(-np.sum(np.abs(obsShapValues), 0))

for topIx in topShap:
    p = PrettierPlot()
    ax = p.makeCanvas()
    
    train.shapDependencePlot(
        obsData=obsData,
        obsShapValues=obsShapValues,
        scatterFeature=featureNames[topIx],
        colorFeature="Age",
        featureNames=featureNames,
        dotSize=50,
        alpha=0.5,
        ax=ax,
    )

###### Summary plots

In [ ]:
# SHAP summary plot
train.shapSummaryPlot(
        obsData=obsData,
        obsShapValues=obsShapValues,
        featureNames=train.data.columns,
    )

##### SHAP values - validation data

###### Force plots - single observations

In [ ]:
# SHAP force plots for individual observations
for i in valid.data.index[:2]:
    valid.singleShapVizTree(obsIx=i, model=model, data=valid.data)

###### Force plots - multiple observations

In [ ]:
# SHAP force plot a set of data
visual = valid.multiShapVizTree(obsIxs=valid.data.index, model=model, data=valid.data)
visual

###### Dependence plots

In [ ]:
# generate SHAP values for set of observations
obsData, _, obsShapValues = valid.multiShapValueTree(
    obsIxs=valid.data.index, model=model, data=valid.data
)

In [ ]:
# SHAP dependence plot grid
gridFeatures = ["Pclass", "Age", "Fare", "SibSp","Parch"]

valid.shapDependenceGrid(
    obsData=obsData,
    obsShapValues=obsShapValues,
    gridFeatures=gridFeatures,
    allFeatures=valid.data.columns,
    dotSize=35,
    alpha=0.5,
)

In [ ]:
# single SHAP dependence plot
p = PrettierPlot()
ax = p.makeCanvas()

valid.shapDependencePlot(
    obsData=obsData,
    obsShapValues=obsShapValues,
    scatterFeature="Age",
    colorFeature="Parch",
    featureNames=valid.data.columns,
    dotSize=50,
    alpha=0.5,
    ax=ax
)


In [ ]:
# SHAP dependence plots for all feature relative to an interaction feature
featureNames = valid.data.columns.tolist()
topShap = np.argsort(-np.sum(np.abs(obsShapValues), 0))

for topIx in topShap:
    p = PrettierPlot()
    ax = p.makeCanvas()
    
    valid.shapDependencePlot(
        obsData=obsData,
        obsShapValues=obsShapValues,
        scatterFeature=featureNames[topIx],
        colorFeature="Age",
        featureNames=featureNames,
        dotSize=50,
        alpha=0.5,
        ax=ax,
    )

###### Summary plots

In [ ]:
# SHAP summary plot
valid.shapSummaryPlot(
        obsData=obsData,
        obsShapValues=obsShapValues,
        featureNames=valid.data.columns,
    )

## Submission - standard models

<a id = 'Submission-standard-models'></a>

In [ ]:
## standard model fit and predict
# select estimator and iteration
# estimator = "lightgbm.LGBMClassifier"; modelIter = 668  #142 survived, 0.77511
# estimator = "xgboost.XGBClassifier"; modelIter = 380  #151 survived, 0.7655
# estimator = "ensemble.RandomForestClassifier"; modelIter = 405  #148 survived, 0.79425
# estimator = "ensemble.GradientBoostingClassifier"; modelIter = 590  #142 survived, 0.7655
estimator = "svm.SVC"; modelIter = 135  #154 survived, 0.755

# extract params and instantiate model
model = train.BayesOptimModelBuilder(
    bayesOptimSummary=bayesOptimSummary, estimator=estimator, modelIter=modelIter
)
model.fit(train.data.values, train.target.values)

# fit model and make predictions
yPred = model.predict(valid.data.values)
print(sum(yPred))

In [ ]:
# generate prediction submission file
submit = pd.DataFrame({"PassengerId": dfValid.PassengerId, "Survived": yPred})
submit.to_csv("submission.csv", index=False)

# Stacking

<a id = 'Stacking'></a>

## Primary models

<a id = 'Primary-models'></a>

In [ ]:
# get out-of-fold predictions
oofTrain, oofValid, columns = train.modelStacker(
    models=topModels,
    bayesOptimSummary=bayesOptimSummary,
    XTrain=train.data.values,
    yTrain=train.target.values,
    XValid=valid.data.values,
    nFolds=10,
    nJobs=10,
)

In [ ]:
# view correlations of predictions
p = PrettierPlot()
ax = p.makeCanvas()
p.prettyCorrHeatmap(
    df=pd.DataFrame(oofTrain, columns=columns), annot=True, ax=ax, vmin=0
)

## Meta model

<a id = 'Meta-model'></a>

In [ ]:
# parameter space
allSpace = {
    "lightgbm.LGBMClassifier": {
        "class_weight": hp.choice("class_weight", [None]),
        "colsample_bytree": hp.uniform("colsample_bytree", 0.4, 0.7),
        "boosting_type": hp.choice("boosting_type", ["dart"]),
        "subsample": hp.uniform("subsample", 0.5, 1),
        "learning_rate": hp.uniform("learning_rate", 0.15, 0.25),
        "max_depth": hp.choice("max_depth", np.arange(4, 20, dtype=int)),
        "min_child_samples": hp.quniform("min_child_samples", 50, 150, 5),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "num_leaves": hp.quniform("num_leaves", 30, 70, 1),
        "reg_alpha": hp.uniform("reg_alpha", 0.75, 1.25),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "subsample_for_bin": hp.quniform("subsample_for_bin", 100000, 350000, 20000),
    },
    "xgboost.XGBClassifier": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.4, 0.7),
        "gamma": hp.quniform("gamma", 0.0, 10, 0.05),
        "learning_rate": hp.quniform("learning_rate", 0.01, 0.2, 0.01),
        "max_depth": hp.choice("max_depth", np.arange(2, 15, dtype=int)),
        "min_child_weight": hp.quniform("min_child_weight", 2.5, 7.5, 1),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "subsample": hp.uniform("subsample", 0.4, 0.7),
    },
    "ensemble.RandomForestClassifier": {
        "bootstrap": hp.choice("bootstrap", [True, False]),
        "max_depth": hp.choice("max_depth", np.arange(2, 10, dtype=int)),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 8000, 10, dtype=int)),
        "max_features": hp.choice("max_features", ["sqrt"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(15, 25, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 20, dtype=int)),
    },
    "ensemble.GradientBoostingClassifier": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 4000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 11, dtype=int)),
        "max_features": hp.choice("max_features", ["sqrt"]),
        "learning_rate": hp.quniform("learning_rate", 0.01, 0.09, 0.01),
        "loss": hp.choice("loss", ["deviance", "exponential"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "svm.SVC": {
        "C": hp.uniform("C", 0.00000001, 15),
        "decision_function_shape": hp.choice("decision_function_shape", ["ovr", "ovo"]),
        "gamma": hp.uniform("gamma", 0.00000001, 1.5),
    },
}

In [ ]:
# execute bayesian optimization grid search
train.execBayesOptimSearch(
    allSpace=allSpace,
    resultsDir="{}_hyperopt_meta_{}.csv".format(rundate, analysis),
    X=oofTrain,
    y=train.target,
    scoring="accuracy",
    nFolds=8,
    nJobs=10,
    iters=1000,
    verbose=0,
)

In [ ]:
# read scores summary table
analysis = "Titanic"
rundate = "20190807"
bayesOptimSummaryMeta = pd.read_csv("{}_hyperopt_meta_{}.csv".format(rundate, analysis))
bayesOptimSummaryMeta[:5]

In [ ]:
# model loss plot
for estimator in np.unique(bayesOptimSummaryMeta["estimator"]):
    train.modelLossPlot(bayesOptimSummary=bayesOptimSummaryMeta, estimator=estimator)

In [ ]:
# estimator parameter plots
for estimator in np.unique(bayesOptimSummaryMeta["estimator"]):
    train.modelParamPlot(
        bayesOptimSummary=bayesOptimSummaryMeta,
        estimator=estimator,
        allSpace=allSpace,
        nIter=100,
        chartProp=15,
    )

## Model performance evaluation - stacked models

<a id = 'Model-performance-evaluation-stacked-models'></a>

In [ ]:
topModels = train.topBayesOptimModels(
    bayesOptimSummary=bayesOptimSummaryMeta, numModels=1
)
topModels

In [ ]:
# classification panel, single model
estimator = "svm.SVC"; modelIter = 135
# estimator = 'ensemble.GradientBoostingClassifier'; modelIter = 590
# estimator = 'xgboost.XGBClassifier'; modelIter = 380

model = train.BayesOptimModelBuilder(
    bayesOptimSummary=bayesOptimSummaryMeta, estimator=estimator, modelIter=modelIter
)

train.classificationPanel(
    model=model, XTrain=oofTrain, yTrain=train.target, labels=[0, 1], nFolds=4
)

In [ ]:
# create classification reports for training data
for estimator, modelIters in topModels.items():
    for modelIter in modelIters:
        model = train.BayesOptimModelBuilder(
            bayesOptimSummary=bayesOptimSummaryMeta,
            estimator=estimator,
            modelIter=modelIter,
        )
        train.classificationPanel(
            model=model, XTrain=oofTrain, yTrain=train.target, labels=[0, 1], nFolds=4
        )

## Submission - stacked models

<a id = 'Submission-stacked-models'></a>

In [ ]:
# best second level learning model
# estimator = "lightgbm.LGBMClassifier"; modelIter = 876 #0.75119
# estimator = "xgboost.XGBClassifier"; modelIter = 821, #0.779
# estimator = "ensemble.RandomForestClassifier"; modelIter = 82 
# estimator = "ensemble.GradientBoostingClassifier"; modelIter = 673 #0.77511
estimator = "svm.SVC"; modelIter = 538 # 0.77511

# extract params and instantiate model
model = train.BayesOptimModelBuilder(
    bayesOptimSummary=bayesOptimSummaryMeta, estimator=estimator, modelIter=modelIter
)

model.fit(oofTrain, train.target.values)
yPred = model.predict(oofValid)
print(sum(yPred))

In [ ]:
# generate prediction submission file
submit = pd.DataFrame({"PassengerId": dfValid.PassengerId, "Survived": yPred})
submit.to_csv("submission.csv", index=False)